In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp '/content/drive/MyDrive/proj/' -r '/content/'

In [3]:
%cd '/content/drive/MyDrive/proj/'

/content/drive/MyDrive/proj


In [4]:
!pwd

/content/drive/MyDrive/proj


In [5]:
!pip install selfcheckgpt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 

In [6]:
!pip install -q flask
!pip install -q pyngrok



In [7]:
import torch
from selfcheckgpt.modeling_mqag import MQAG

AUTH_TOKEN = '2dkugw4dTR56DEO3moeiiHZTcYc_55prtiLmU8ck6FTXYSrRR'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

mqag_model = MQAG(
    g1_model_type='race', # race (more abstractive), squad (more extractive)
    device=device
)

cuda
MQAG (race) initialized to cuda


In [8]:
import numpy as np

def generate_questions_answers(context):
  questions = mqag_model.generate(context=context, do_sample=True, num_questions=10)
  final_set = []
  for i, question_item in enumerate(questions):
      question = question_item['question']
      options = question_item['options']

      single_question = [{'question': question, 'options': options}]
      probs = mqag_model.answer(questions=single_question, context=context)
      correct_answer = options[np.argmax(probs)]

      final_set.append({'question': question, 'options': options, 'answer': correct_answer})

  return final_set

In [ ]:
from flask import Flask, render_template, request, redirect, url_for , jsonify
import json
from pyngrok import ngrok, conf

conf.get_default().auth_token = AUTH_TOKEN



app = Flask(__name__)
# run_with_ngrok(app)


global answers
global quiz_data_v2
global questionText
questionText = ""


with open("details.json", "r") as jf:
    data = json.load(jf)
    users = data["login"]


@app.route('/', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        # Retrieve form data
        email = request.form.get('login-email')
        password = request.form.get('login-password')

        if email in users.keys() and users[email]==password:
            return redirect(url_for('generate_qna'))

    return render_template('auth.html')


@app.route('/signup', methods=['GET', 'POST'])
def signup():
    if request.method == 'POST':
        # Retrieve form data
        email = request.form.get('signup-email')
        password = request.form.get('signup-password')
        password2 = request.form.get('confirm-signup-password')

        # Perform validation or other actions with the form data
        with open("details.json", "w") as jfk:
            users[email] = password
            json.dump(users, jfk)

        # Print form data (for demonstration purposes)
        print(f'Email: {email}, Password: {password} ,  password2 : { password2}')

        # Add your logic for user registration or other actions
        return redirect(url_for('login'))

    return render_template('auth.html')




@app.route('/create_post' , methods=['GET', 'POST'] )
def create_post():
    # Additional logic for generating Q&A or render a template
     if request.method == 'POST':
            # Retrieve form data
        new_post = request.form.get('post-input')


        # Perform validation or other actions with the form data

        # Print form data (for demonstration purposes)
        print(f'new_post: {new_post}')
        return redirect(url_for('forum'))




@app.route('/generate_qna' , methods=['GET', 'POST'])
def generate_qna():
    # Additional logic for generating Q&A or render a template
    question_data = [
    {"subject": "Math", "num_questions": 10},
    {"subject": "Science", "num_questions": 10},
    # Add more data as needed
]

    return render_template('generate_qna.html' , questions=question_data)


@app.route('/answer_evaluation')
def answer_evaluation():

    subj_data = data["scores"]
    subjects_data = [
        {"subject": list(subj_data.keys())[0], "score": subj_data[list(subj_data.keys())[0]]},
        {"subject": list(subj_data.keys())[1], "score": subj_data[list(subj_data.keys())[1]]},
        {"subject": list(subj_data.keys())[2], "score": subj_data[list(subj_data.keys())[2]]},
    ]

    return render_template('answer_evaluation.html', subjects_data=subjects_data)
    # Additional logic for generating Q&A or render a template



@app.route('/forum')
def forum():
    # Additional logic for generating Q&A or render a template
    post_data = [
        {
            'title' : "Golanginya",
            'content'  : "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Consequat aliquet maecenas ut sit nulla"
        },
        {
            'title' : "Golanginya",
            'content'  : "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Consequat aliquet maecenas ut sit nulla"
        },
        {
            'title' : "Golanginya",
            'content'  : "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Consequat aliquet maecenas ut sit nulla"
        }
    ]
    return render_template('forum.html' ,post_data=post_data )


@app.route('/quiz' , methods=['GET', 'POST'])
def quiz():
    global answers
    global quiz_data_v2
    global questionText
    quiz_data = [

    {
        "question": "For Fiscal year 2021, what was your total GhG Carbon Emission for all scopes?",
        "options": [
            "Scope 1 - Determine environmental impact levels.",
            "Scope 2 - Reduce carbon footprints.",
            "Scope 3 - Enhance environmental impacts on a larger scale.",
            "I do not know the answer to this question.",
        ],
    },
    {
        "question": "Another question?",
        "options": [
            "Option 1",
            "Option 2",
            "Option 3",
            "I do not know the answer to this question.",
        ],
    },


    # Add more questions and answers as needed
    ]
    if request.method == 'POST':
        print("post method ************")
        questionText = request.form.get('questionText')
        print(f"Launching quiz for subject: {questionText}")
        quiz_data = generate_questions_answers(questionText.replace('\n', ' '))

        quiz_data_v2 = []
        answers = []
        for item in quiz_data:
            # d = ast.literal_eval(item)
            js_data = {
                "question": item['question'],
                "answers": item['options']
            }
            quiz_data_v2.append(js_data)
            answers.append(item['answer'])
        print(quiz_data_v2)
        print("Answers:", answers)

        # return redirect(url_for('quiz'))
        render_template('quiz.html' , quiz_data=quiz_data_v2)

    return render_template('quiz.html', quiz_data=quiz_data_v2)


@app.route('/quiz/response', methods=['POST'])
def quiz_response():
    global quiz_data_v2
    global answers

    if request.method == 'POST':
        # Retrieve the selected answers from the form
        selected_answers = []

        for question in quiz_data_v2:
            selected_answers.append(request.form.get(f'selected-answer-{question["question"]}'))


        # You can now process the selected answers as needed
        # For example, you might want to store them in a database or perform some analysis

        # For now, let's just print the selected answers
        print("Selected Answers:", selected_answers)
        score = 0
        quiz_data_ans = []

        for i in range(len(selected_answers)):
          quiz_data_ans.append({'question': quiz_data_v2[i]['question'], 'answer':answers[i]})
          if selected_answers[i]==answers[i]:
            score += 10


        result = "Fail" if score<50 else "Pass"


        # quiz_data_ans = [

        #         {
        #             "question": "For Fiscal year 2021, what was your total GhG Carbon Emission for all scopes?",
        #         "answer": "Scope 3 - Enhance environmental impacts on a larger scale."
        #     },
        #     {
        #         "question": "Another question?",
        #         "answer": "Option 3"
        #     },

        # ]

        # You can redirect to a different page or render a new template
        return render_template('QuizResult.html', result=result , score=score , quiz_data_ans=quiz_data_ans )
        # You might want to render a response or redirect to another page
    return render_template('quiz.html', quiz_data=quiz_data_v2)


@app.route('/logout')
def logout():
    # Additional logic for generating Q&A or render a template
    return redirect(url_for('login'))


ngrok_tunnel = ngrok.connect(5000)
print(ngrok_tunnel)
# print('Running at:', ngrok_tunnel.public_url)

if __name__ == '__main__':
    app.run(port=5000)


NgrokTunnel: "https://e34f-34-125-105-172.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
